# Prototype a simple calculation on cloud data with coiled dask workers

## Notes

- How do I authenticate my coiled token in a script?

I can do `coiled login -t <token>`, but that is not safe?

Probably have to parse this file somehow?
`/home/jovyan/.config/dask/coiled.yaml`

- Why cant coiled install `cmip6_pp` from conda? Is this a channel issue? 
    There are also version conflicts there. Might be mamba or that coiled is running their own channel?
    
    **SOLUTION**: Oh neat, you can just pass the path to an [environment file](https://docs.coiled.io/user_guide/software_environment_creation.html#conda)!
    
- I am not sure if the problem was transient, or if this matters. But when I built the environment locally the remote build process seems to go a lot faster (might have also been that the docker image already existed and was just updated?)

In [6]:
import coiled

In [7]:
# close out all running clusters
for name in coiled.list_clusters().keys():
    coiled.delete_cluster(name=name)

In [8]:
coiled.create_software_environment(
    conda='environment.yml',
)
# Create a Dask cluster which uses 
# software environment
cluster = coiled.Cluster(software="cmip6_derived_cloud_datasets", n_workers=30)#, 

Updating software environment...
Found existing software environment build, returning


Output()

Found software environment build


In [9]:
# Connect Dask to that cluster
import dask.distributed
client = dask.distributed.Client(cluster)
print("Dask Dashboard:", client.dashboard_link)

Dask Dashboard: http://ec2-18-116-231-200.us-east-2.compute.amazonaws.com:8787


In [10]:
# for AWS S3:
import intake
col = intake.open_esm_datastore("https://cmip6-pds.s3.amazonaws.com/pangeo-cmip6.json")

tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <zmq.eventloop.ioloop.ZMQIOLoop object at 0x7f9bec790550>>, <Task finished name='Task-202' coro=<Cluster._close() done, defined at /srv/conda/envs/cmip6_derived_cloud_datasets/lib/python3.9/site-packages/coiled/cluster.py:646> exception=ValueError('No cluster available to close!')>)
Traceback (most recent call last):
  File "/srv/conda/envs/cmip6_derived_cloud_datasets/lib/python3.9/site-packages/tornado/ioloop.py", line 741, in _run_callback
    ret = callback()
  File "/srv/conda/envs/cmip6_derived_cloud_datasets/lib/python3.9/site-packages/tornado/ioloop.py", line 765, in _discard_future_result
    future.result()
  File "/srv/conda/envs/cmip6_derived_cloud_datasets/lib/python3.9/site-packages/coiled/cluster.py", line 648, in _close
    raise ValueError("No cluster available to close!")
ValueError: No cluster available to close!
/srv/conda/envs/cmip6_derived_cloud_dat

In [11]:
# choose a sample dataset
cat = col.search(
    source_id='GFDL-CM4',
    variable_id='thetao',
    grid_label='gn',
    experiment_id='historical'
)
ddict = cat.to_dataset_dict(
    zarr_kwargs={
        'consolidated':True,
        'use_cftime':True
    }
)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [12]:
ds = ddict['CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn']
ds

<xarray.Dataset>
Dimensions:    (bnds: 2, lev: 35, member_id: 1, time: 1980, vertex: 4, x: 1440, y: 1080)
Coordinates:
  * bnds       (bnds) float64 1.0 2.0
    lat        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lat_bnds   (y, x, vertex) float32 dask.array<chunksize=(1080, 1440, 4), meta=np.ndarray>
  * lev        (lev) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03
    lev_bnds   (lev, bnds) float64 dask.array<chunksize=(35, 2), meta=np.ndarray>
    lon        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lon_bnds   (y, x, vertex) float32 dask.array<chunksize=(1080, 1440, 4), meta=np.ndarray>
  * time       (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
    time_bnds  (time, bnds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
  * x          (x) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
  * y          (y) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
  * member_id  (member_id) <U8 'r1i1p1f1'
Dimensions without coordinates: vertex
Data variables:
    thetao     (member_id, time, lev, y, x) float32 dask.array<chunksize=(1, 1, 35, 1080, 1440), meta=np.ndarray>
Attributes: (12/51)
    Conventions:             CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:             CMIP
    branch_method:           standard
    branch_time_in_child:    0.0
    branch_time_in_parent:   36500.0
    comment:                 <null ref>
    ...                      ...
    variant_label:           r1i1p1f1
    status:                  2019-08-06;created;by nhn2@columbia.edu
    netcdf_tracking_ids:     hdl:21.14100/46c8bd6a-8787-403d-85cb-e4e52c237fa...
    version_id:              v20180701
    intake_esm_varname:      ['thetao']
    intake_esm_dataset_key:  CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn

In [23]:
# set up the pangeo scratch bucket
# not sure that works with the credentials in this env
import os
PANGEO_SCRATCH = os.environ['PANGEO_SCRATCH']
# -> gs://pangeo-scratch/<username>
import fsspec
mapper = fsspec.get_mapper(f'{PANGEO_SCRATCH}/cmip6_derived_cloud_data.zarr')
# mapper can now be to read / write zarr stores

In [24]:
data_test = ds.mean(['x','y'])

In [ ]:
data_test.to_zarr(mapper, mode='w')